# Part 1 - Working with Text Data

### Use Python string methods remove irregular whitespace from the following string:

In [22]:
whitespace_string = "\n\n  This is a    string   that has  \n a lot of  extra \n   whitespace.   "
print(whitespace_string)



  This is a    string   that has  
 a lot of  extra 
   whitespace.   


In [23]:
import re

In [24]:
 ##### Your Code Here #####
whitespace_string = whitespace_string.strip()
whitespace_string = re.sub(r'[\n]', '',whitespace_string)
whitespace_string= " ".join(whitespace_string.split())
print(whitespace_string)


This is a string that has a lot of extra whitespace.


### Use Regular Expressions to take the dates in the following .txt file and put them into a dataframe with columns for:

[RegEx dates.txt](https://github.com/ryanleeallred/datasets/blob/master/dates.txt)

- Day
- Month
- Year


In [4]:
!wget https://raw.githubusercontent.com/ryanleeallred/datasets/master/dates.txt

--2019-03-29 16:29:58--  https://raw.githubusercontent.com/ryanleeallred/datasets/master/dates.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.200.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.200.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 296 [text/plain]
Saving to: ‘dates.txt.1’

dates.txt.1         100%[===================>]     296  --.-KB/s    in 0s      

2019-03-29 16:29:58 (78.9 MB/s) - ‘dates.txt.1’ saved [296/296]



In [43]:
##### Your Code Here #####

with open('dates.txt', 'r', encoding='utf-8') as f:
    contents = f.read()
  
contents

'March 8, 2015\nMarch 15, 2015\nMarch 22, 2015\nMarch 29, 2015\nApril 5, 2015\nApril 12, 2015\nApril 19, 2015\nApril 26, 2015\nMay 3, 2015\nMay 10, 2015\nMay 17, 2015\nMay 24, 2015\nMay 31, 2015\nJune 7, 2015\nJune 14, 2015\nJune 21, 2015\nJune 28, 2015\nJuly 5, 2015\nJuly 12, 2015\nJuly 19, 2015'

In [44]:
contents = re.sub('[,]','', contents)

In [45]:
contents

'March 8 2015\nMarch 15 2015\nMarch 22 2015\nMarch 29 2015\nApril 5 2015\nApril 12 2015\nApril 19 2015\nApril 26 2015\nMay 3 2015\nMay 10 2015\nMay 17 2015\nMay 24 2015\nMay 31 2015\nJune 7 2015\nJune 14 2015\nJune 21 2015\nJune 28 2015\nJuly 5 2015\nJuly 12 2015\nJuly 19 2015'

In [46]:
dates = re.split('\\n',contents)

In [47]:
dates = [item.split(" ") for item in dates]

In [48]:
dates = pd.DataFrame(dates, columns = ["March","day","year"])

In [49]:
dates

,March,day,year
0,March,8,2015
1,March,15,2015
2,March,22,2015
3,March,29,2015
4,April,5,2015
5,April,12,2015
6,April,19,2015
7,April,26,2015
8,May,3,2015
9,May,10,2015


# Part 2 - Bag of Words 

### Use the twitter sentiment analysis dataset found at this link for the remainder of the Sprint Challenge:

[Twitter Sentiment Analysis Dataset](https://raw.githubusercontent.com/ryanleeallred/datasets/master/twitter_sentiment_binary.csv)

 ### Clean and tokenize the documents ensuring the following properties of the text:

1) ~Text should be lowercase.~

2) Stopwords should be removed.

3) Punctuation should be removed.

4) Tweets should be tokenized at the word level. 

(The above don't necessarily need to be completed in that specific order.)

### Output some cleaned tweets so that we can see that you made all of the above changes.


In [1]:
import pandas as pd

In [4]:
import re

In [5]:
##### Your Code Here #####
df = pd.read_csv("https://raw.githubusercontent.com/ryanleeallred/datasets/master/twitter_sentiment_binary.csv")

In [6]:
df["clean_text"] = df["SentimentText"].str.lower()
df["clean_text"] = df["clean_text"].str.replace("&lt;","<")
df["clean_text"] = df["clean_text"].str.replace("&gt;","<")
df["clean_text"] = df["clean_text"].apply(lambda x : re.sub(r"[.,@#!?]","",x))

In [7]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /home/ec2-user/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/ec2-user/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [8]:
from nltk.tokenize import word_tokenize

In [9]:
df["token_text"] = df["clean_text"].apply(word_tokenize)

In [10]:
df['token_text'].head()

0                  [is, so, sad, for, my, apl, friend]
1                 [i, missed, the, new, moon, trailer]
2                      [omg, its, already, 7:30, :, o]
3    [omgaga, im, sooo, im, gunna, cry, i, 've, bee...
4        [i, think, mi, bf, is, cheating, on, me, t_t]
Name: token_text, dtype: object

### How should TF-IDF scores be interpreted? How are they calculated?

#### Your Answer Here #####

the scores try to measure what is unique about a document. TF is term frequency. IDF is inverse document frequency. TF-IDF tries to measure The frequency of terms in a document with respect to the infrequency of the term in the set of documents.

# Part 3 - Document Classification

1) Use Train_Test_Split to create train and test datasets.

2) Vectorize the tokenized documents using your choice of vectorization method. 

 - Stretch goal: Use both of the methods that we talked about in class.

3) Create a vocabulary using the X_train dataset and transform both your X_train and X_test data using that vocabulary.

4) Use your choice of binary classification algorithm to train and evaluate your model's accuracy. Report both train and test accuracies.

 - Stretch goal: Use an error metric other than accuracy and implement/evaluate multiple classifiers.



In [12]:
##### Your Code Here #####
!pip install -U gensim
import gensim

    100% |████████████████████████████████| 24.2MB 2.1MB/s eta 0:00:01
Requirement not upgraded as not directly required: six>=1.5.0 in /home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages (from gensim) (1.11.0)
Requirement not upgraded as not directly required: scipy>=0.18.1 in /home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages (from gensim) (1.1.0)
    100% |████████████████████████████████| 40kB 27.6MB/s ta 0:00:01
Requirement not upgraded as not directly required: numpy>=1.11.3 in /home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages (from gensim) (1.15.4)
Requirement not upgraded as not directly required: boto>=2.32 in /home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages (from smart-open>=1.7.0->gensim) (2.48.0)
Requirement not upgraded as not directly required: requests in /home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages (from smart-open>=1.7.0->gensim) (2.20.1)
Requirement not upgraded as not directly required

In [13]:
import nltk
#nltk.download('all')

In [14]:
from sklearn.model_selection import train_test_split
X = df["clean_text"]
y=df["Sentiment"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [95]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(max_features=10000, ngram_range=(1,2), stop_words='english')
train_word_counts = vectorizer.fit_transform(X_train)

In [96]:
X_train_vectorized = pd.DataFrame(train_word_counts.toarray(), columns=vectorizer.get_feature_names())

In [97]:
X_test_vectorized = pd.DataFrame(vectorizer.transform(X_test).toarray(), columns = vectorizer.get_feature_names())

In [98]:
X_train_vectorized.shape

(79991, 10000)

In [99]:
from sklearn.linear_model import LogisticRegression

LR = LogisticRegression(random_state=42).fit(X_train_vectorized, y_train)

train_predictions = LR.predict(X_train_vectorized)
test_predictions = LR.predict(X_test_vectorized)

from sklearn.metrics import accuracy_score

print(f'Train Accuracy: {accuracy_score(y_train, train_predictions)}')
print(f'Test Accuracy: {accuracy_score(y_test, test_predictions)}')

Train Accuracy: 0.8016901901463914
Test Accuracy: 0.7485748574857486


In [100]:
roc_auc_score(y_test, test_predictions)

0.737241851249746

In [75]:
!pip install -i https://test.pypi.org/simple/lambdata-crawftv

ERROR: You must give at least one requirement to install (see "pip help install")
You are using pip version 10.0.1, however version 19.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [76]:
from lambdata_crawftv.ClassificationVisualization import classification_visualization

ModuleNotFoundError: No module named 'lambdata_crawftv'

In [77]:
from lambdata_crawftv.ClassificationVisualization import classification_visualization
classification_visualization(y_test,test_predictions)

ModuleNotFoundError: No module named 'lambdata_crawftv'

In [53]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [89]:
tfidf_vectorizer = TfidfVectorizer(max_features=10000, ngram_range=(1,2), stop_words='english')
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

In [91]:
LR = LogisticRegression(random_state=42).fit(X_train_tfidf, y_train)

train_predictions = LR.predict(X_train_tfidf)
test_predictions = LR.predict(X_test_tfidf)

print(f'Train Accuracy: {accuracy_score(y_train, train_predictions)}')
print(f'Test Accuracy: {accuracy_score(y_test, test_predictions)}')


Train Accuracy: 0.8001900213774049
Test Accuracy: 0.7493249324932493


In [92]:
from sklearn.metrics import roc_auc_score

roc_auc_score(y_test, test_predictions)

0.7383528043080675

# Part 4 -  Word2Vec

1) Fit a Word2Vec model on your cleaned/tokenized twitter dataset. 

2) Display the 10 words that are most similar to the word "twitter"

In [18]:
from gensim.models.word2vec import Word2Vec
model = Word2Vec(df["token_text"], min_count=5, size=100)

In [21]:
##### Your Code Here #####
model.most_similar("twitter")

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:2: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  from ipykernel import kernelapp as app


[('facebook', 0.7547707557678223),
 ('everyone', 0.6824592351913452),
 ('fb', 0.6778935194015503),
 ('myspace', 0.6669214963912964),
 ('someone', 0.6316429376602173),
 ('list', 0.6132799386978149),
 ('youtube', 0.6023457050323486),
 ('updates', 0.5994023084640503),
 ('blog', 0.5865471363067627),
 ('comments', 0.5779016017913818)]

In [101]:
model.vocabulary_

AttributeError: 'Word2Vec' object has no attribute 'vocabulary_'